[This helped me a lot to learn also](https://www.kaggle.com/code/none00000/csiro)    <<-- Click Me

In [1]:
import warnings
warnings.filterwarnings('ignore')


import gc
import os
import time
import timm
import torch
import pickle
import numpy as np
import pandas as pd

from PIL import Image
import torch.nn as nn
from tqdm import tqdm

import torch.optim as optim
from sklearn.metrics import r2_score
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import TensorDataset, DataLoader, Dataset
from sklearn.metrics import mean_squared_error, mean_absolute_error
from albumentations import Compose, Resize, Normalize, HorizontalFlip, VerticalFlip

In [4]:
class CONFIG:
    SEED = 67

    TRAIN_PATH = '/kaggle/input/csiro-biomass/train.csv'
    TEST_PATH =  '/kaggle/input/csiro-biomass/test.csv'
    MODEL_NAME = 'convnext_tiny'

    device = 'cuda' if torch.cuda.is_available() else 'cpu'

    BATCH_SIZE =  4
    NUM_WORKERS = 2
    N_FOLDS = 5

    FREEZE_EPOCHS = 1
    LEARNING_RATE = 1e-4
    FINETUNE_LR = 1e-5

    EPOCHS = 2
    
    loss_weights = {
        "Dry_Green_g": 0.2,
        "Dry_Total_g": 0.5,
        "GDM_g": 0.3
    }

    weights = {
        "Dry_Clover_g": 0.1,
        "Dry_Dead_g": 0.1,
        "Dry_Green_g": 0.1,
        "Dry_Total_g": 0.5,
        "GDM_g": 0.2
    }

    IMG_SIZE = 768

cfg = CONFIG()

In [5]:
class Transform:
    def __init__(self):
        self.pipeline = self.__make_pipeline()

    def __make_pipeline(self):
        base_transforms = [
            Resize(cfg.IMG_SIZE, cfg.IMG_SIZE),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]
            ),
            ToTensorV2()
        ]

        original_view = Compose([
            *base_transforms
        ])

        hflip_view = Compose([
            HorizontalFlip(p=1.0),
            *base_transforms
        ])

        vflip_view = Compose([
            VerticalFlip(p=1.0),
            *base_transforms
        ])
        
        return [original_view, hflip_view, vflip_view]

In [6]:
class BiomassDataset(Dataset):
    def __init__(self, df, transform, train=True):
        self.train = train
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        path = self.df['image_path'].iloc[idx]
        img = np.array(Image.open(f'/kaggle/input/csiro-biomass/{path}').convert("RGB"))

        mid = img.shape[0] // 2
        left, right = img[:, :mid], img[:, mid:]
        
        transform_left = self.transform(image=left)['image']
        transform_right = self.transform(image=right)['image']
        
        if self.train:
            targets = torch.tensor(self.df[['Dry_Green_g', 'Dry_Total_g', 'GDM_g']].iloc[idx].to_numpy(), dtype=torch.float)
            all_targets = torch.tensor(self.df[['Dry_Clover_g', 'Dry_Dead_g', 'Dry_Green_g', 'Dry_Total_g', 'GDM_g']].iloc[idx].to_numpy(), dtype=torch.float)
            return transform_left, transform_right, targets, all_targets
        else:
            return transform_left, transform_right

In [7]:
def weighted_r2_torch(y_true, y_pred, w):
    w = torch.tensor(list(w.values()), dtype=torch.float32)
    w = w / w.sum()
    
    y_bar = (w * y_true).sum(dim=1, keepdim=True)
    ss_res = (w * (y_true - y_pred) ** 2).sum()
    ss_tot = (w * (y_true - y_bar) ** 2).sum()
    return 1 - ss_res / ss_tot

def competition_score(all_preds_3, all_targets_5):
    
    pred_green = all_preds_3['green']
    pred_total = all_preds_3['total']
    pred_gdm = all_preds_3['gdm']

    pred_clover = np.maximum(0, pred_gdm - pred_green)
    pred_dead = np.maximum(0, pred_total - pred_gdm)

    y_preds = np.stack([
        pred_clover,
        pred_dead,
        pred_green,
        pred_total,
        pred_gdm
    ], axis=1)

    y_true = all_targets_5

    r2_scores = r2_score(y_true, y_preds, multioutput='raw_values')

    weighted_r2_total = 0.0
    for i, weight in enumerate(cfg.weights.values()):
        weighted_r2_total += r2_scores[i] * weight

    return weighted_r2_total

In [8]:
def clean_ids(data):
    return data.split('__')[0]
    
def preprocessing(data):
    data['sample_id'] = data['sample_id'].apply(clean_ids)

    if 'target' in data.columns:
        return data.pivot_table(
            index=[
                'sample_id',
                'image_path'
            ],
                columns='target_name', 
                values='target'
            ).reset_index()

    data = data[['sample_id', 'image_path']]
    return data.drop_duplicates()

In [9]:
class BioModel(nn.Module):
    def __init__(self, model_name, pretrained, n_targets=3, drop_rate=0.3):
        super(BioModel, self).__init__()
        self.backbone =  timm.create_model(
            model_name,
            pretrained=pretrained,
            num_classes=0,
            global_pool='avg'
        )

        self.n_features = self.backbone.num_features
        self.n_combined_features = self.n_features * 2


        self.head_total = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(self.n_combined_features // 2, 1) 
        )

        self.head_gdm = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(self.n_combined_features // 2, 1) 
        )
        
        self.head_green = nn.Sequential(
            nn.Linear(self.n_combined_features, self.n_combined_features // 2),
            nn.ReLU(),
            nn.Dropout(drop_rate),
            nn.Linear(self.n_combined_features // 2, 1)
        )

    def forward(self, left, right):
        features_left = self.backbone(left)
        features_right = self.backbone(right)

        combined = torch.cat([features_left, features_right], dim=1)
        out_total = self.head_total(combined)
        out_gdm = self.head_gdm(combined)
        out_green = self.head_green(combined)

        return out_total, out_gdm, out_green

In [10]:
class WeightedLoss(nn.Module):
    def __init__(self, loss_weights_dict):
        super(WeightedLoss, self).__init__()
        
        self.criterion = nn.SmoothL1Loss()
        self.weights = loss_weights_dict

    def forward(self, predictions, targets):
        pred_green, pred_total, pred_gdm = predictions

        true_green = targets[:, 0].unsqueeze(-1)
        true_total   = targets[:, 1].unsqueeze(-1)
        true_gdm = targets[:, 2].unsqueeze(-1)

        loss_total = self.criterion(pred_total, true_total)
        loss_gdm   = self.criterion(pred_gdm, true_gdm)
        loss_green = self.criterion(pred_green, true_green)

        total_loss = (
            self.weights['Dry_Green_g'] * loss_green +
            self.weights['GDM_g'] * loss_gdm +
            self.weights['Dry_Total_g'] * loss_total
        )

        return total_loss

In [11]:
class EarlyStopping:
    def __init__(self, patience=7, min_delta=0, verbose=False, filename='fold'):
        self.patience = patience
        self.min_delta = min_delta
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.filename = filename

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score > self.best_score + self.min_delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model...')
        torch.save(model.state_dict(), self.filename)
        self.val_loss_min = val_loss

In [12]:
pipeline = Transform().pipeline[0]
train = pd.read_csv(cfg.TRAIN_PATH)
train = preprocessing(train)
train['fold'] = -1

# test = pd.read_csv(cfg.TEST_PATH)
# test = preprocessing(test)
if len(train) > 100:
    num_bins = 10

train['total_bin'] = pd.cut(train['Dry_Total_g'], bins=num_bins, labels=False)

skf = StratifiedKFold(
    n_splits=cfg.N_FOLDS, 
    shuffle=True, 
    random_state=cfg.SEED
)

for fold_num, (train_idx, valid_idx) in enumerate(skf.split(train, train['total_bin'])):
    train.loc[valid_idx, 'fold'] = fold_num

In [13]:
def train_one_epoch(model, loader, criterion, optimizer, device=cfg.device):
    model.train()  
    epoch_loss = 0.0
    
    pbar = tqdm(loader, desc="Training", leave=False)
    for (img_left, img_right, train_targets, _all_targets_ignored) in pbar:
        
        img_left = img_left.to(device)
        img_right = img_right.to(device)
        targets = train_targets.to(device)
        
        predictions = model(img_left, img_right)
        optimizer.zero_grad()
        
        loss = criterion(predictions, targets)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        pbar.set_postfix(loss=f'{loss.item():.4f}')
        
    return epoch_loss / len(loader)

def validate_one_epoch(model, loader, criterion, device=cfg.device):
    model.eval()
    epoch_loss = 0.0
    
    all_preds_3 = {'total': [], 'gdm': [], 'green': []}
    all_targets_list = []

    with torch.no_grad():
        pbar = tqdm(loader, desc="Validating", leave=False)
        for (img_left, img_right, train_targets, all_targets) in pbar:
            
            img_left = img_left.to(device)
            img_right = img_right.to(device)
            train_targets = train_targets.to(device)
            
            pred_green, pred_total, pred_gdm = model(img_left, img_right)
            
            predictions_tuple = (pred_total, pred_gdm, pred_green)
            loss = criterion(predictions_tuple, train_targets)
            epoch_loss += loss.item()
            
            all_preds_3['total'].append(pred_total.cpu().numpy())
            all_preds_3['gdm'].append(pred_gdm.cpu().numpy())
            all_preds_3['green'].append(pred_green.cpu().numpy())
            all_targets_list.append(all_targets.cpu().numpy())


    preds_dict_np = {
        'total': np.concatenate(all_preds_3['total']).flatten(),
        'gdm':   np.concatenate(all_preds_3['gdm']).flatten(),
        'green': np.concatenate(all_preds_3['green']).flatten()
    }
    targets_np_5 = np.concatenate(all_targets_list)
    
    score = competition_score(preds_dict_np, targets_np_5)
    
    avg_epoch_loss = epoch_loss / len(loader)
    
    return avg_epoch_loss, score

In [14]:
def run_fold(fold):
    print(f"\n{'='*50}")
    print(f" Fold: {fold}")
    print(f"{'='*50}")
    
    start_time = time.time()
    
    train_df = train[train['fold'] != fold].reset_index(drop=True)
    valid_df = train[train['fold'] == fold].reset_index(drop=True)
    
    train_dataset = BiomassDataset(train, pipeline)
    valid_dataset = BiomassDataset(valid_df, pipeline)
    
    train_loader = DataLoader(
        train_dataset, batch_size=cfg.BATCH_SIZE, shuffle=True,
        num_workers=cfg.NUM_WORKERS, pin_memory=True
    )
    
    valid_loader = DataLoader(
        valid_dataset, batch_size=cfg.BATCH_SIZE * 2, shuffle=False,
        num_workers=cfg.NUM_WORKERS, pin_memory=True
    )
    
    print(f"MODEL:  '{cfg.MODEL_NAME}'...")
    model_base = BioModel(cfg.MODEL_NAME, cfg.MODEL_NAME)
    
    if torch.cuda.device_count() > 1:
        print(f" {torch.cuda.device_count()} GPU avilable")
        model = nn.DataParallel(model_base)
    else:
        model = model_base
        
    model.to(cfg.device)
    
    criterion = WeightedLoss(cfg.loss_weights).to(cfg.device)
    
    print(f"Epochs: {cfg.EPOCHS} | FREEZE : {cfg.FREEZE_EPOCHS} | LR: {cfg.LEARNING_RATE}")

    for param in model.backbone.parameters():
        param.requires_grad = False
        
    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()), 
        lr=cfg.LEARNING_RATE
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.1, patience=2 
    )
    
    early_stop = EarlyStopping(filename=f'{cfg.MODEL_NAME}_fold_{fold}.pt')

    for epoch in range(1, cfg.FREEZE_EPOCHS + 1):
        print(f"\n--- Epoch {epoch}/{cfg.EPOCHS} ---")
        
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
        valid_loss, score = validate_one_epoch(model, valid_loader, criterion)
        
        scheduler.step(valid_loss)
        
        print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | Score (R^2): {score:.4f}")
        
        early_stop(score, model)
        if early_stop.early_stop:
            print(f'Stopping training on epoch {epoch} with best score {early_stop.best_score}')
            break
        

    print(f"\n--- Fine-tuning  ---")
    print(f"Epochs: {cfg.FREEZE_EPOCHS + 1}/{cfg.EPOCHS} | LR: {cfg.FINETUNE_LR}")

    for param in model.backbone.parameters():
        param.requires_grad = True
        
    optimizer = optim.Adam(
        model.parameters(), 
        lr=cfg.FINETUNE_LR
    )
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.2, patience=3
    )
    
    for epoch in range(cfg.FREEZE_EPOCHS + 1, cfg.EPOCHS + 1):
        print(f"\n--- Epoch {epoch}/{cfg.EPOCHS} ---")
        
        train_loss = train_one_epoch(model, train_loader, criterion, optimizer)
        valid_loss, score = validate_one_epoch(model, valid_loader, criterion)
        
        scheduler.step(valid_loss)
        
        print(f"Epoch {epoch} - Train Loss: {train_loss:.4f} | Valid Loss: {valid_loss:.4f} | Score (R^2): {score:.4f}")
        
        early_stop(score, model)
        if early_stop.early_stop:
            print(f'Stopping training on epoch {epoch} with best score {early_stop.best_score}')
            break
            
    end_time = time.time()
    print(f"\nFold {fold} runs in  {(end_time - start_time)/60:.2f}")
    print(f"Best Score : {early_stop.best_score:.4f}")
    
    del model, train_loader, valid_loader, train_dataset, valid_dataset
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
try:
    for i in range(cfg.N_FOLDS):
        run_fold(i)
        break
except Exception as e:
    gc.collect()
    torch.cuda.empty_cache()
    raise e


 Fold: 0
MODEL:  'convnext_tiny'...


model.safetensors:   0%|          | 0.00/114M [00:00<?, ?B/s]

Epochs: 2 | FREEZE : 1 | LR: 0.0001

--- Epoch 1/2 ---


Epoch 1 - Train Loss: 23.1091 | Valid Loss: 20.5305 | Score (R^2): 0.0888

--- Fine-tuning  ---
Epochs: 2/2 | LR: 1e-05

--- Epoch 2/2 ---


Training:  21%|██        | 19/90 [00:44<02:44,  2.32s/it, loss=11.5117]